In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [1]:
import os
import numpy as np
import pickle
import keras
import tensorflow as tf
import warnings
from matplotlib import pyplot as plt
from IPython.display import clear_output

from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout, Flatten, LSTM, Embedding, Activation, Add
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU, ReLU, ELU
from keras.regularizers import l2 # L2-regularisation
from keras.models import Model
from keras import optimizers
from keras.layers.advanced_activations import LeakyReLU

warnings.filterwarnings("ignore")

Using TensorFlow backend.


### Load data

In [ ]:
# def load_data(data_name):
#     pkl_file = open(data_name + '.pkl', 'rb')
#     data = pickle.load(pkl_file)
#     pkl_file.close()
#     return data

# print('Loading data...')
# folder = '../input/pairwise2540/'

# train = load_data(folder + 'machine_train')
# train_label = list(load_data(folder + 'machine_train_label'))
# test = load_data(folder + 'machine_test')
# test_label = list(load_data(folder + 'machine_test_label'))
# valid = load_data(folder + 'machine_valid')
# valid_label = list(load_data(folder + 'machine_valid_label'))
# print('Data are loaded')

In [2]:
def load_data(data_name):
    pkl_file = open(data_name + '.pkl', 'rb')
    data = pickle.load(pkl_file)
    pkl_file.close()
    return data

print('Loading data...')
folder = '../input/aminodataset/'

train = np.array(load_data(folder + 'train'))
train_label = load_data(folder + 'train_label')
test = np.array(load_data(folder + 'test'))
test_label = load_data(folder + 'test_label')
valid = np.array(load_data(folder + 'valid'))
valid_label = load_data(folder + 'valid_label')
print('Data are loaded')

Loading data...


FileNotFoundError: [Errno 2] No such file or directory: '../input/neuraldataset2540/neural_train.pkl'

In [ ]:
np.shape(train), np.shape(train_label)

## ResNet

In [ ]:
def relu_bn(inputs):
    relu = (ELU(alpha=0.1))(inputs)
    
    return relu


def residual_block_identity(x, filters, l2_lambda):
    
    f1, f2 = filters
    
    y = (Conv2D(filters = f1, 
                kernel_size = (5, 5),
                activation = 'linear', 
                W_regularizer = l2(l2_lambda), 
                padding = 'same'))(x)
    
    y = (LeakyReLU(alpha=0.1))(y)
    
    y = (Conv2D(filters = f2, 
                kernel_size = (5, 5), 
                activation = 'linear', 
                W_regularizer = l2(l2_lambda), 
                padding = 'same'))(y)
    
    y = (LeakyReLU(alpha=0.1))(y)
    
    out = (Add())([x, y])
    
    out = (MaxPooling2D(pool_size = (2, 2),
                        padding = 'same'))(out)
    
    out = (Dropout(0.3))(y)
    
    return out



def residual_block_convolutional(x, l2_lambda):
            
    y = (Conv2D(filters = 16, 
               kernel_size = (1, 1),
               strides = (2, 2),
               activation = 'linear'))(x)
    y = relu_bn(y)
    
    y = (LeakyReLU(alpha=0.1))(y)
    
    y = (Conv2D(filters = 32, 
               kernel_size = (5, 5),
               padding = "same", 
               activation ='linear'))(y)
    y = relu_bn(y)

    y = (LeakyReLU(alpha=0.1))(y)
    
    x = (Conv2D(filters = 32, 
               kernel_size = (1, 1),
               strides = (2, 2),
               padding = "same", 
               activation ='linear'))(x)
    
    y = (LeakyReLU(alpha=0.1))(y)
        
    out = (Add())([x, y])
    out = (MaxPooling2D(pool_size = (2, 2), 
                        padding = 'same'))(out)
    out = (Dropout(0.1))(out)
    
    return out

In [ ]:
def resnet(out_shape, mm, input_shape = (64, 64, 3), classes = 2):
    
    l2_lambda = 0.0001
    num_filters = np.array([16, 32])
    num_blocks_list = [2, 4]
    
    inputs = Input(input_shape)
    
    t = (BatchNormalization())(inputs)
    t = (Conv2D(kernel_size = (mm, 5),
               strides = (1, 1),
               filters = 32,
               padding = "same"))(inputs)
        
    for i in range(len(num_blocks_list)):
        
        num_blocks = num_blocks_list[i]
        t = residual_block_convolutional(inputs, l2_lambda = l2_lambda)
        
        for j in range(num_blocks):
            
            t = residual_block_identity(t, filters = num_filters, l2_lambda = l2_lambda)
    
    layer4 = (Flatten())(t)
    layer4 = (BatchNormalization())(layer4)
    layer4 = (Dense((5 * mm), 
                    activation = 'linear', 
                    W_regularizer = l2(l2_lambda)))(layer4)
    layer4 = (LeakyReLU(alpha = 0.1))(layer4)
    layer4 = (Dropout(0.1))(layer4)
    finish = []
    
    for i in range(int((mm ** 2 - mm) / 2)):
        output = Dense(out_shape, activation = 'softmax', name = 'main_output'+str(i))(layer4)
        finish.append(output)

    model = Model(inputs = [inputs], outputs = finish)
    model.compile(loss = "categorical_crossentropy", 
                  optimizer=keras.optimizers.Adam(lr = 0.001))

    return model

In [ ]:
batch_size = 8
epochs = 25
mm = 40

model = resnet(np.shape(train_label)[2], mm, input_shape = (train.shape[1], train.shape[2], 1))

print('Start training...')
history = model.fit(train, list(train_label), batch_size = batch_size, epochs = epochs, verbose = 1,
                          validation_data=(np.array(valid), list(valid_label)))

In [ ]:
predicted = model.predict(test)

print(np.shape(predicted))
print(np.shape(test_label))

output = open('predicted' + '.pkl', 'wb')
pickle.dump(predicted, output)
output.close()

output = open('test' + '.pkl', 'wb')
pickle.dump(test_label, output)
output.close()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('resnet')

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import operator
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score


def score(test_label, predicted, metric):
    
    predicted = np.array(predicted).reshape(780 * 73, 2)
    test_label = np.array(test_label).reshape(780 * 73, 2)
    
    id_score = {}
    pred = []
    test = []
    
    L = []
    L_2 = []
    L_5 = []
    L_10 = []
    
    for i in range(len(predicted)):
        id_score[i] = predicted[i][1]
        
    for key, value in reversed(sorted(id_score.items(), key=operator.itemgetter(1))):
        pred.append(np.argmax([1 - value, value]))
        test.append(np.argmax(test_label[key]))
        
    if metric == 'recall':
        
        L.append(recall_score(test, pred))
        L_2.append(recall_score(test[:len(test) // 2], pred[:len(pred) // 2]))
        L_5.append(recall_score(test[:len(test) // 5], pred[:len(pred) // 5]))
        L_10.append(recall_score(test[:len(test) // 10], pred[:len(pred) // 10]))
        
    elif metric == 'f1':
        
        L.append(f1_score(test, pred))
        L_2.append(f1_score(test[:len(test) // 2], pred[:len(pred) // 2]))
        L_5.append(f1_score(test[:len(test) // 5], pred[:len(pred) // 5]))
        L_10.append(f1_score(test[:len(test) // 10], pred[:len(pred) // 10]))
        
    elif metric == 'precision':
        
        L.append(precision_score(test, pred))
        L_2.append(precision_score(test[:len(test) // 2], pred[:len(pred) // 2]))
        L_5.append(precision_score(test[:len(test) // 5], pred[:len(pred) // 5]))
        L_10.append(precision_score(test[:len(test) // 10], pred[:len(pred) // 10]))
        
    else:
        
        raise ValueError('No metric was found')
            
    return L, L_2, L_5, L_10

In [ ]:
def print_results(test_label, proba):
    
    # f1 score
    
    L, L_2, L_5, L_10 = score(test_label, proba, 'f1')
    print('F1 score:')
    print('L:', L)
    print('L_2:', L_2)
    print('L_5:', L_5)
    print('L_10:', L_10)
    
    # precision score
    
    L, L_2, L_5, L_10 = score(test_label, proba, 'precision')
    print('Precision score:')
    print('L:', L)
    print('L_2:', L_2)
    print('L_5:', L_5)
    print('L_10:', L_10)
    
    
    # recall score
    
    L, L_2, L_5, L_10 = score(test_label, proba, 'recall')
    print('Recall score:')
    print('L:', L)
    print('L_2:', L_2)
    print('L_5:', L_5)
    print('L_10:', L_10)

In [ ]:
print_results(test_label, predicted)

In [ ]:
# from keras.utils import plot_model
# from IPython.display import Image
# from matplotlib.pyplot import savefig

# model = resnet(np.shape(train_label)[2], mm, input_shape = (train.shape[1], train.shape[2], 1))
# plot_model(model, show_shapes=True, show_layer_names=True, to_file='lstm_arch.png')

# # Image(retina=True, filename='model2.png')